In [26]:
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

import numpy as np
import scipy.stats as st
import pandas as pd
import os
import warnings
import iqplot

import bokeh.io
import bokeh.plotting
import bokeh.layouts
bokeh.io.output_notebook()

Loading BokehJS ...

### Specifying the data, models, main effects

In [27]:
filename = 'MotorBehaviordata_GFSPF_Livia_111323.xlsx'
data_path = os.path.join("..", "..", "Data", "GFxSPF_ASO", filename)
sheet_name = 'Pole and Beam'

# Type of test for current analysis
mtest = 'Beam_T'

# Group factors
effect_1 = 'Genotype' 
effect_2 = 'Microbiome'

# Whether or not the experiment has several cohorts and a dedicated cohort column
cohort_col = False 

### Plotting characteristics

Specifying palettes and plotting order.

In [28]:
# Palette:
palette_CI = ["#bdbdbd", "#000000", "#6baed6", "#08519c"]

### Tidying up the data

In [29]:
pole_beam_df = pd.read_excel(data_path, sheet_name=sheet_name)
pole_beam_df = pole_beam_df.rename(columns={'ID ':'ID', 'Group ':'Group', 'treatment':'Treatment'})

if (effect_1 not in pole_beam_df.columns) or (effect_2 not in pole_beam_df.columns):
    pole_beam_df[[effect_1, effect_2]] = pole_beam_df['Group'].str.split('-', expand = True)

Creating the working data frame only with the values of interest, removing missing values and cases where mice jumped off of the pole/beam.

In [30]:
pole_beam_df = pole_beam_df.replace(to_replace = 'jumped', value = float("NaN"))

res = [i for i in list(pole_beam_df.columns) if (mtest) in i]

if cohort_col:
    col_list = [cohort_col, effect_1, effect_2, 'ID']
else:
    col_list = [effect_1, effect_2, 'ID']

beam_df = pole_beam_df[col_list + res].copy()
beam_df_long = beam_df.melt(id_vars = col_list, var_name='Trial', value_name='Time')
beam_df_long['Trial'] = beam_df_long['Trial'].str[-2:]
beam_df_long['ID'] = beam_df_long['ID'].astype('str')
beam_df_long = beam_df_long.dropna()

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_54473/3630996893.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pole_beam_df = pole_beam_df.replace(to_replace = 'jumped', value = float("NaN"))


In [32]:
p = iqplot.stripbox(
    data=beam_df_long,
    q='Time',
    cats=['Microbiome', 'Genotype'],
    order=[
        ("SPF", "WT"),
        ("SPF", "ASO"),
        ("GF", "WT"),
        ("GF", "ASO")
    ],
    q_axis='y',
    palette=palette_CI,
    spread="jitter",
    tooltips=[
        ("Genotype", "@Genotype"),
        ("Microbiome", "@Microbiome"),
        ("Mouse ID", "@ID"),
        ("Trial #", "@Trial")
    ]
)

p.output_backend = 'svg'
bokeh.io.export_svgs(p, filename=os.path.join("..", "Output", "gfspf_beam_all_trials.svg"))
bokeh.io.show(p)

In [34]:
%load_ext watermark
%watermark -v -p numpy,scipy,pandas,scikit_posthocs,iqplot,bebi103,bokeh,jupyterlab

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.13.2
IPython version      : 9.0.2

numpy          : 2.1.3
scipy          : 1.15.2
pandas         : 2.2.3
scikit_posthocs: 0.11.4
iqplot         : 0.3.7
bebi103        : 0.1.26
bokeh          : 3.7.0
jupyterlab     : 4.3.6

